<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/BubbleDewCalculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00


In [23]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
from scipy.interpolate import Akima1DInterpolator
jax.config.update("jax_enable_x64", True)
from plotly.subplots import make_subplots
from scipy.optimize import root
from scipy.optimize import root_scalar

In [3]:
p=Props(["Benzene","Toluene"])

In [4]:
def bubbleP_ideal(x, T):
    Pi = x*p.Pvap(T)
    P = np.sum(Pi)
    return P, Pi/P

In [5]:
z=np.array([0.3, 0.7])
z

array([0.3, 0.7])

In [6]:
bubbleP_ideal(z, 373.15)

(Array(105789.27018675, dtype=float64),
 Array([0.51014734, 0.48985266], dtype=float64))

In [7]:
def bubbleT_ideal(x, P):

    def P_dev(T):
        return bubbleP_ideal(x,T)[0] - P

    T=root(P_dev, 300.).x[0]
    return T,x*p.Pvap(T)/P


In [8]:
bubbleT_ideal(z, 101325.)

(371.6543967389883, Array([0.51137952, 0.48862048], dtype=float64))

In [9]:
def dewP_ideal(y, T):
    P=1./(np.sum(y/p.Pvap(T)))
    return P, y*P/p.Pvap(T)

In [10]:
dewP_ideal(z, 373.15)

(Array(89901.78980933, dtype=float64),
 Array([0.14992484, 0.85007516], dtype=float64))

In [11]:

def dewT_ideal(y, P):

    def P_dev(T):
        return dewP_ideal(y, T)[0] - P
    T = root(P_dev, 300.).x[0]

    return T,  y*P/p.Pvap(T)

In [12]:
dewT_ideal(z, 89901.79)

(373.15000007098485, Array([0.14992484, 0.85007516], dtype=float64))

In [13]:
z1 = np.linspace(0.,1.,20)
zs=np.c_[z1, 1-z1]

In [14]:
x1s=[]
y1s=[]
Ps=[]
T=373.15
for z in zs:
    P, y =bubbleP_ideal(z, T)
    x1s.append(z[0])
    y1s.append(y[0])
    Ps.append(P)

fig=make_subplots()
fig.add_scatter(x=x1s, y=Ps, name='Bubble')
fig.add_scatter(x=y1s, y=Ps, name='Dew')
fig.update_layout(width=600, height=400, template='plotly_dark',
                  title=f'Benzene (1)/Toluene (2) T={T}')

In [15]:
x1s=[]
y1s=[]
Ts=[]
P=101325.
for z in zs:
    T, y =bubbleT_ideal(z, P)
    x1s.append(z[0])
    y1s.append(y[0])
    Ts.append(T)

fig=make_subplots()
fig.add_scatter(x=x1s, y=Ts, name='Bubble')
fig.add_scatter(x=y1s, y=Ts, name='Dew')
fig.update_layout(width=600, height=400, template='plotly_dark',
                  title=f'Benzene (1)/Toluene (2) P={P}')

In [16]:
fig3=make_subplots()
fig3.add_scatter(x=x1s, y=y1s)
fig3.add_scatter(x=[0.,1.],y=[0.,1.], mode='lines', line_color='rgb(50,50,50)')
fig3.update_layout(width=400,height=400,template='plotly_dark', showlegend=False)

In [17]:
p1, p2= p.Pvap(373.15)
alpha = p1/p2

In [18]:
x1plot = np.linspace(0,1,50)
y1plot = alpha*x1plot/((1-x1plot) + alpha*x1plot)

In [19]:
fig4=make_subplots()
fig4.add_scatter(x=x1s, y=y1s, mode='markers')
fig4.add_scatter(x=x1plot, y=y1plot, mode='lines')
fig4.add_scatter(x=[0.,1.],y=[0.,1.], mode='lines', line_color='rgb(50,50,50)')
fig4.update_layout(width=400,height=400,template='plotly_dark', showlegend=False)

In [20]:
fig5=make_subplots()
fig5.add_scatter(x=x1s, y=y1s, mode='markers')
fig5.add_scatter(x=x1plot, y=y1_spline(x1plot), mode='lines')
fig5.add_scatter(x=[0.,1.],y=[0.,1.], mode='lines', line_color='rgb(50,50,50)')
fig5.update_layout(width=400,height=400,template='plotly_dark', showlegend=False)

NameError: name 'y1_spline' is not defined

In [21]:
x1_spline=Akima1DInterpolator(y1s,x1s)

In [ ]:
x1_spline(0.85)

array(0.69047878)

In [22]:
#Rachford-Rice

#Benzene/Toluene, 1 mol/s of feed (0.4,0.6) Benzene/Toluene flows into a flash drum (T=370K, P=101325), what are the flow rates
#and compositions of the vapor and liquid products.

In [38]:

p=Props(['Benzene','Toluene','pXylene'])
z=np.array([0.1,0.3,0.6])


In [39]:
def flash_idealPT(z, P, T):

    K=p.Pvap(T)/P
    def rachford(VF):
        return np.sum(z*(K-1)/(VF*(K-1) +1))

    res=root_scalar(rachford, bracket=(0,1))
    VF = res.root
    x=z/(1-VF + K *VF)
    y=K*x
    return x, y, VF

In [40]:
bubbleP_ideal(z, 373)

(Array(59153.0701055, dtype=float64),
 Array([0.30288185, 0.37374352, 0.32337464], dtype=float64))

In [41]:
dewP_ideal(z, 373)

(Array(42645.66389017, dtype=float64),
 Array([0.0238026 , 0.17360668, 0.80259072], dtype=float64))

In [43]:
flash_idealPT(z, (59153+42645)/2, 373)

(Array([0.05483313, 0.26169186, 0.68347501], dtype=float64),
 Array([0.19301201, 0.37888784, 0.42810015], dtype=float64),
 0.32687246715754475)